In [45]:
import os

In [46]:
%pwd

'd:\\software\\python_vs\\lone_approval_prediction'

In [47]:
os.chdir("../")

In [48]:
%pwd

'd:\\software\\python_vs'

In [49]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/rppawar491/Lone_approval_prediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "rppawar491"
os.environ["MLFLOW_TRACKING_PASSWARD"] = "024579b40f9cfcb8f0cb1be07a0ed82e8ae6f12e"

#### Entity

In [50]:
from pathlib import Path
from dataclasses import dataclass

@dataclass
class ModelEvaluationConfig:
    root_dir : Path
    x_test_data_path : Path
    y_test_data_path : Path
    model_path : Path
    all_params : dict
    metric_file_name : Path
    mlflow_uri : str


#### Configuration Manager

In [51]:
from lone_approval_prediction.constants import *
from lone_approval_prediction.utils.comman import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.logisticRegressor
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            x_test_data_path = config.x_test_data_path,
            y_test_data_path = config.y_test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            mlflow_uri = "https://dagshub.com/rppawar491/Lone_approval_prediction.mlflow"
        )
        
        return model_evaluation_config

#### Component

In [52]:
import mlflow
import joblib
import pandas as pd
from urllib.parse import urlparse
from sklearn.metrics import accuracy_score, precision_score, f1_score

class ModelEvaluation:
    def __init__(self, config):
        self.config = config

    def eval_metrics(self, actual, pred):
        try:
            accuracy = accuracy_score(actual, pred)
            precision = precision_score(actual, pred, average='weighted')
            f1 = f1_score(actual, pred, average='weighted')
            return accuracy, precision, f1
        except Exception as e:
            print(f"Error calculating metrics: {e}")
            return 0, 0, 0

    def log_into_mlflow(self):
        try:
            test_data = pd.read_csv(self.config.x_test_data_path)
            model = joblib.load(self.config.model_path)

            test_x = test_data
            test_y = pd.read_csv(self.config.y_test_data_path)

            # Set the MLflow tracking URI and pass the access token
            mlflow.set_registry_uri(self.config.mlflow_uri)
            mlflow.set_tracking_uri(self.config.mlflow_uri)

            # Ensure the tracking URI is properly set
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
            print(f"Tracking URI: {mlflow.get_tracking_uri()}")

            with mlflow.start_run():
                predicted_classes = model.predict(test_x)

                accuracy, precision, f1 = self.eval_metrics(test_y, predicted_classes)
                scores = {"accuracy": accuracy, "precision": precision, "f1": f1}

                mlflow.log_params(self.config.all_params)
                mlflow.log_metrics(scores)

                # Model registry does not work with file store
                if tracking_url_type_store != "file":
                    mlflow.sklearn.log_model(model, "model", registered_model_name="LogisticRegression")
                else:
                    mlflow.sklearn.log_model(model, "model")

                return scores

        except Exception as e:
            raise e


In [53]:
try :
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'